# Oscilloskope utility – using Ethernet with GPS timemarks 

In [1]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run "Record" on the oscilloscope
# and wait for 508 frames to be acquired.
# Step 2:
# Run this cell to initialize grabbing.


# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
        
# Default oscilloscope record timeout [s]
loop_sleep_time = 60
        
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::10.1.1.254::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::10.1.1.254::INSTR
RIGOL TECHNOLOGIES,DS2102A,DS2D174501960,00.03.04.SP2


## Read repeatedly records from oscilloscope

In [2]:
filename = 1
    
if (filename == 1):
    for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
        print 'Deleting', f
        os.remove(f)
else:
    print 'Not removing old files, as filename {0} is not 1.'.format(filename)


osc.write(':STOP') # start recording
time.sleep(0.5)
    
while True:
    #print('  Enter to continue.')
    #raw_input() Wait for key press

    osc.write(':FUNC:WREC:OPER REC') # start recording
    run_start_time = time.time()
    print '  Capturing...'
    time.sleep(0.5)
    
    while True:
        osc.write(':FUNC:WREC:OPER?') # finish recording?
        reply = osc.read()
        if reply == 'STOP':
            run_time = round(time.time() - run_start_time, 2)
            print('  Subrun finished, capturing for %.2f seconds.' % run_time)
            break
        time.sleep(0.01)

    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print 'XINC:', xinc,
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print 'YINC:', yinc,
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print 'TRIG:', trig,
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print 'YORIGIN:', yorig,
    osc.write(':WAVeform:XORigin?')
    xorig = float(osc.read(100))
    print 'XORIGIN:', xorig,
    

    osc.write(':FUNC:WREP:FEND?') # get number of last frame
    frames = int(osc.read(100))
    print 'FRAMES:', frames, 'SUBRUN', filename
    
    with h5py.File('./data/data'+'{:02.0f}'.format(filename)+'_'+str(int(round(time.time(),0)))+'.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames)) # write number of frames
        hf.create_dataset('XINC', data=(xinc)) # write axis parameters
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        hf.create_dataset('XORIGIN', data=(xorig))
        hf.create_dataset('CAPTURING', data=(run_time))
        osc.write(':FUNC:WREP:FCUR 1') # skip to n-th frame
        time.sleep(0.5)
        for n in range(1,frames+1):
            osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
            time.sleep(0.001)

            osc.write(':WAV:DATA?') # read data
            #time.sleep(0.4)
            wave1 = bytearray(osc.read_raw(500))
            wave2 = bytearray(osc.read_raw(500))
            wave3 = bytearray(osc.read_raw(500))
            #wave4 = bytearray(osc.read(500))
            #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
            wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
            hf.create_dataset(str(n), data=wave)
    filename = filename + 1
    

Deleting ./data/data06_1511961541.h5
Deleting ./data/data24_1511967767.h5
Deleting ./data/data16_1511965252.h5
Deleting ./data/data15_1511964921.h5
Deleting ./data/data13_1511964266.h5
Deleting ./data/data28_1511969218.h5
Deleting ./data/data05_1511961074.h5
Deleting ./data/data04_1511960593.h5
Deleting ./data/data22_1511967133.h5
Deleting ./data/data10_1511963152.h5
Deleting ./data/data01_1511959176.h5
Deleting ./data/data09_1511962768.h5
Deleting ./data/data21_1511966824.h5
Deleting ./data/data20_1511966522.h5
Deleting ./data/data30_1511970360.h5
Deleting ./data/data25_1511968088.h5
Deleting ./data/data03_1511960050.h5
Deleting ./data/data07_1511961969.h5
Deleting ./data/data02_1511959616.h5
Deleting ./data/data17_1511965576.h5
Deleting ./data/data14_1511964600.h5
Deleting ./data/data08_1511962359.h5
Deleting ./data/data18_1511965898.h5
Deleting ./data/data11_1511963530.h5
Deleting ./data/data19_1511966209.h5
Deleting ./data/data23_1511967451.h5
Deleting ./data/data31_1511970704.h5
D

KeyboardInterrupt: 